In [3]:
# some imports
import os
import numpy as np
import pickle
from functools import reduce
import operator
from glob import glob
%matplotlib notebook
from matplotlib import pyplot as plt
from natsort import natsorted
import keras
from keras.optimizers import Adam

# import custom functions and viewing tools
from VisTools import multi_slice_viewer0, mask_viewer0
from KerasModel import BlockModel, dice_coef_loss, CalcVolumes

ModuleNotFoundError: No module named 'keras'

In [2]:
#~# some parameters to set for inference #~#
# model version to use
model_version = 6
model_weights_path = os.path.join(os.getcwd(),
                                  'BestModelWeights_dataset2_v{:02d}.h5'.format(model_version))

NameError: name 'os' is not defined

In [47]:
# load testing data
with open('test_input_files_v{:02d}.pkl'.format(model_version), 'rb') as f:
    input_files_test_groups = pickle.load(f)
test_input_files = natsorted(reduce(operator.add,input_files_test_groups))
with open('test_target_files_v{:02d}.pkl'.format(model_version), 'rb') as f:
    target_files_test_groups = pickle.load(f)
test_target_files = natsorted(reduce(operator.add,target_files_test_groups))
# find unique sequence names
sequences = [os.path.split(os.path.dirname(f))[1] for f in test_input_files]
unq_sequences = list(set(sequences))
# count up number of subjects and scans for each sequence
seq_subj_count = [[len(g) for g in input_files_test_groups if s in reduce(operator.add,g)] for s in unq_sequences]
seq_subj_sum = [len(g) for g in seq_subj_count]
seq_scan_sum = [np.sum(g) for g in seq_subj_count]
print('The testing set contains:')
for i,s in enumerate(unq_sequences):
    print('-----------------------')
    print('{} {} unique subjects'.format(seq_subj_sum[i],s))
    print('{} {} scans'.format(seq_scan_sum[i],s))
# resplit by sequence instead of by subject
grouped_input_files = [[f for f in test_input_files if s in f] for s in unq_sequences]
grouped_target_files = [[f for f in test_target_files if s in f] for s in unq_sequences]

The testing set contains:
-----------------------
10 SSFSE_output unique subjects
23 SSFSE_output scans
-----------------------
10 opposed_output unique subjects
10 opposed_output scans
-----------------------
10 t1nfs_output unique subjects
19 t1nfs_output scans


In [ ]:
print('Loading data...')
inputs = [np.concatenate([np.load(f) for f in g])[...,np.newaxis] for g in grouped_input_files]
targets = [np.concatenate([np.load(f) for f in g])[...,np.newaxis] for g in grouped_target_files]
print("Data loaded")

In [ ]:
# prepare model
model = BlockModel(inputs[0].shape,filt_num=16,numBlocks=4)
model.compile(optimizer=Adam(), loss=dice_coef_loss)

# load best weights
model.load_weights(model_weights_path)
# evaluate on test data
print('Evaluating data...')
for i,d in enumerate(unq_sequences):
    score = model.evaluate(inputs[i],targets[i],verbose=1)
    print("Test Dice score on {} is {:.03f}".format(d,1-score))

In [ ]:
# Get predicted masks
output = [model.predict(inp,batch_size=16) for inp in inputs]

In [ ]:
# display image of index of choice
seq = 0
ind = 0
DisplayDifferenceMask(testX[ind,...,0],testY[ind,...,0],output[ind,...,0])

In [ ]:
# display some results
seq_num = 1
print('Generating {} masks...'.format(unq_sequences[seq_num]))
mask_viewer0(inputs[seq_num][...,0],targets[seq_num][...,0],output[seq_num][...,0],name=unq_sequences[seq_num])
plt.show()

Below section not yet fully updated for dataset2 analysis

In [ ]:
# set voxel dimensions (in cm)
sx = 1
sy = 1
sz = 1
vox_vol = sx*sy*sz
# loop over different sequence types
for sequence in all_inputs:
    # loop over all subjects
    results = []
    for inp_file,targ_file in zip(all_inputs,all_targets):
        # run function to perform calculations
        calc,truth = CalcVolumes(inp_file,targ_file,vox_vol,model)
        results.append((calc,truth))
# display results
print('---------------------------------------')
print(inp_file[-30:])
print("The calculated volume is {:.02f} voxels".format(calc))
print("The actual volume is {:.02f} voxels".format(truth))

In [1]:
x = [r[1] for r in results]
y = [r[0] for r in results]
plt.figure()
plt.plot(x,y,'ro')
plt.plot(np.arange(0,np.max(x)),np.arange(0,np.max(x)),'k--')
plt.xlim([0,np.max(x)])
plt.ylim([0,np.max(y)])
plt.ylabel('Calculated Volume (cc)')
plt.xlabel('Actual Volume (cc)')
plt.title("Correlation of Actual and Predicted Liver Volumes of Test Subjects")
plt.show()
cc = np.corrcoef(x,y)[0,1]
print('Correlation coefficient is {:.03f}'.format(cc))

NameError: name 'results' is not defined